# Training the models
1. [Introduction](#introduction)
2. [Some paragraph](#paragraph1)
    1. [Sub paragraph](#subparagraph1)
3. [Another paragraph](#paragraph2)

## import required packages

In [28]:
import stan_jupyter as stan # this is a version of pystan that works with jupyter notebook, otherwise use `import stan`
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle
import json

## define experiment and model names <a name="introduction"></a>

In [29]:
ALL_EXPERIMENTS = ['memory', 'memory2', 'memory_color_short', 'memory_color_long']
ALL_MODELS = ['var_beta', 'invar_beta']

## define stan code for candidate models

### model code 1: beta varies with x

In [30]:
var_beta_model_code = '''
functions {
  vector get_theta(int N, int[] subj, vector time, vector x, vector mu0, vector sigma0, vector beta0, vector alpha0, vector delta_mu, vector delta_sigma, vector delta_beta) {
      
    vector[N] mu; // prep mean
    vector[N] sigma; // prep sd
    vector[N] beta; // p(corr | prep)
    vector[N] alpha; // p(corr | not prep)
  
    real phi; // p(prep | t)
    real not_phi; // p(not prep | t)
    vector[2] psi;  // p(corr | prep) & p(corr | not prep)
  
    vector[N] theta; // p(corr | t)
  
    for (n in 1:N) {
  
      mu[n]    = inv_logit(mu0[subj[n]] + delta_mu[subj[n]] * x[n]);
      sigma[n] = inv_logit(sigma0[subj[n]] + delta_sigma[subj[n]] * x[n]);
      beta[n]  = inv_logit(beta0[subj[n]] + delta_beta[subj[n]] * x[n]);
      alpha[n] = inv_logit(alpha0[subj[n]]);
  
      phi = normal_cdf(time[n], mu[n], sigma[n]);
      not_phi = 1 - phi;

      psi[1] = not_phi * alpha[n];
      psi[2] = phi * beta[n];
  
      theta[n] = sum(psi);  // response prob (@t)
    }
    return(theta);
  }
  
} 
data {
  
  int<lower=0> N; // n observations
  int<lower=0> J; // n subjects
  int<lower=0,upper=1> y[N]; // observations
  int<lower=0> subj[N]; // subject ids
  vector<lower=0,upper=1>[N] time; // time in seconds
  
  vector[N] x; //covariate
  
  int<lower=0,upper=1> gq;
  int<lower=0,upper=1> prior_only;
  
} 
parameters {
  real<lower=0> mu_scale;
  real<lower=0> sigma_scale;
  real<lower=0> beta_scale;
  real<lower=0> alpha_scale;
  real<lower=0> delta_mu_scale;
  real<lower=0> delta_sigma_scale;
  real<lower=0> delta_beta_scale;
    
  real mu_loc;
  real sigma_loc;
  real beta_loc;
  real alpha_loc;
  real delta_mu_loc;
  real delta_sigma_loc;
  real delta_beta_loc;
  
  vector[J] mu_raw;
  vector[J] sigma_raw;
  vector[J] beta_raw;
  vector[J] alpha_raw;
  vector[J] delta_mu_raw;
  vector[J] delta_sigma_raw;
  vector[J] delta_beta_raw;

}
transformed parameters {
  vector[N] theta;
  vector[J] mu0;
  vector[J] sigma0;
  vector[J] beta0;
  vector[J] alpha0;
  vector[J] delta_mu;
  vector[J] delta_sigma;
  vector[J] delta_beta;
  
  // subj level intercepts
  mu0 = mu_raw * mu_scale + mu_loc;
  sigma0 = sigma_raw * sigma_scale + sigma_loc;
  beta0 = beta_raw * beta_scale + beta_loc;
  alpha0 = alpha_raw * alpha_scale + alpha_loc;
  
  // subj level slopes
  delta_mu = delta_mu_raw * delta_mu_scale + delta_mu_loc;
  delta_sigma = delta_sigma_raw * delta_sigma_scale + delta_sigma_loc;
  delta_beta = delta_beta_raw * delta_beta_scale + delta_beta_loc;
  
  // probability of correct reponse at each time given parameters
  theta = get_theta(N, subj, time, x, mu0, sigma0, beta0, alpha0, delta_mu, delta_sigma, delta_beta);
} 
model {

  // priors on group-level locations and scales of intercepts
  mu_loc ~ normal(-.5, .5);
  sigma_loc ~ normal(-2, .5);
  beta_loc ~ normal(2, .5);
  alpha_loc ~ normal(-1, .5);
  
  mu_scale ~ normal(0, .5);
  sigma_scale ~ normal(0, .5);
  beta_scale ~ normal(0, .5);
  alpha_scale ~ normal(0, .5);
  
  // priors on *raw* subj-level intercepts
  mu_raw ~ normal(0, 1);
  sigma_raw ~ normal(0, 1);
  beta_raw ~ normal(0, 1);
  alpha_raw ~ normal(0, 1);
  
  // priors on group-level locations and scales of slopes
  delta_mu_loc ~ normal(0, .5);
  delta_sigma_loc ~ normal(0, .5);
  delta_beta_loc ~ normal(0, .5);
  
  delta_mu_scale ~ normal(0, .5);
  delta_sigma_scale ~ normal(0, .5);
  delta_beta_scale ~ normal(0, .5);
  
  // priors on *raw* subj-level slopes
  delta_mu_raw ~ normal(0, 1);
  delta_sigma_raw ~ normal(0, 1);
  delta_beta_raw ~ normal(0, 1);
  
  if (prior_only == 0) {
    y ~ bernoulli(theta);
  }
  
} 
generated quantities {
  
  vector[N * gq] log_lik;
  
  if (gq == 1) {
  
    for (n in 1:N) {
  
      log_lik[n] = bernoulli_lpmf(y[n] | theta[n]);
  
    }
  }
} 

'''

### model code 2: beta does not vary with x

In [31]:
invar_beta_model_code = '''
functions {
  vector get_theta(int N, int[] subj, vector time, vector x, vector mu0, vector sigma0, vector beta0, vector alpha0, vector delta_mu, vector delta_sigma) {
      
    vector[N] mu; // prep mean
    vector[N] sigma; // prep sd
    vector[N] beta; // p(corr | prep)
    vector[N] alpha; // p(corr | not prep)
  
    real phi; // p(prep | t)
    real not_phi; // p(not prep | t)
    vector[2] psi;  // p(corr | prep) & p(corr | not prep)
  
    vector[N] theta; // p(corr | t)
  
    for (n in 1:N) {
  
      mu[n]    = inv_logit(mu0[subj[n]] + delta_mu[subj[n]] * x[n]);
      sigma[n] = inv_logit(sigma0[subj[n]] + delta_sigma[subj[n]] * x[n]);
      beta[n]  = inv_logit(beta0[subj[n]]);
      alpha[n] = inv_logit(alpha0[subj[n]]);
  
      phi = normal_cdf(time[n], mu[n], sigma[n]);
      not_phi = 1 - phi;

      psi[1] = not_phi * alpha[n];
      psi[2] = phi * beta[n];
  
      theta[n] = sum(psi);  // response prob (@t)
    }
    return(theta);
  }
  
} 
data {
  
  int<lower=0> N; // n observations
  int<lower=0> J; // n subjects
  int<lower=0,upper=1> y[N]; // observations
  int<lower=0> subj[N]; // subject ids
  vector<lower=0,upper=1>[N] time; // time in seconds
  
  vector[N] x; //covariate
  
  int<lower=0,upper=1> gq;
  int<lower=0,upper=1> prior_only;
  
} 
parameters {
  real<lower=0> mu_scale;
  real<lower=0> sigma_scale;
  real<lower=0> beta_scale;
  real<lower=0> alpha_scale;
  real<lower=0> delta_mu_scale;
  real<lower=0> delta_sigma_scale;
    
  real mu_loc;
  real sigma_loc;
  real beta_loc;
  real alpha_loc;
  real delta_mu_loc;
  real delta_sigma_loc;
  
  vector[J] mu_raw;
  vector[J] sigma_raw;
  vector[J] beta_raw;
  vector[J] alpha_raw;
  vector[J] delta_mu_raw;
  vector[J] delta_sigma_raw;

}
transformed parameters {
  vector[N] theta;
  vector[J] mu0;
  vector[J] sigma0;
  vector[J] beta0;
  vector[J] alpha0;
  vector[J] delta_mu;
  vector[J] delta_sigma;
  
  // subj level intercepts
  mu0 = mu_raw * mu_scale + mu_loc;
  sigma0 = sigma_raw * sigma_scale + sigma_loc;
  beta0 = beta_raw * beta_scale + beta_loc;
  alpha0 = alpha_raw * alpha_scale + alpha_loc;
  
  // subj level slopes
  delta_mu = delta_mu_raw * delta_mu_scale + delta_mu_loc;
  delta_sigma = delta_sigma_raw * delta_sigma_scale + delta_sigma_loc;
  
  // probability of correct reponse at each time given parameters
  theta = get_theta(N, subj, time, x, mu0, sigma0, beta0, alpha0, delta_mu, delta_sigma);
} 
model {

  // priors on group-level locations and scales of intercepts
  mu_loc ~ normal(-.5, .5);
  sigma_loc ~ normal(-2, .5);
  beta_loc ~ normal(2, .5);
  alpha_loc ~ normal(-1, .5);
  
  mu_scale ~ normal(0, .5);
  sigma_scale ~ normal(0, .5);
  beta_scale ~ normal(0, .5);
  alpha_scale ~ normal(0, .5);
  
  // priors on *raw* subj-level intercepts
  mu_raw ~ normal(0, 1);
  sigma_raw ~ normal(0, 1);
  beta_raw ~ normal(0, 1);
  alpha_raw ~ normal(0, 1);
  
  // priors on group-level locations and scales of slopes
  delta_mu_loc ~ normal(0, .5);
  delta_sigma_loc ~ normal(0, .5);
  
  delta_mu_scale ~ normal(0, .5);
  delta_sigma_scale ~ normal(0, .5);
  
  // priors on *raw* subj-level slopes
  delta_mu_raw ~ normal(0, 1);
  delta_sigma_raw ~ normal(0, 1);
  
  if (prior_only == 0) {
    y ~ bernoulli(theta);
  }
  
} 
generated quantities {
  
  vector[N * gq] log_lik;
  
  if (gq == 1) {
  
    for (n in 1:N) {
  
      log_lik[n] = bernoulli_lpmf(y[n] | theta[n]);
  
    }
  }
} 

'''

In [32]:
model_code = {
    'var_beta': var_beta_model_code,
    'invar_beta': invar_beta_model_code
}

# example: one dataset, one model

## choose dataset and model

In [33]:
exp_name = ALL_EXPERIMENTS[0]
model_name = ALL_MODELS[1]

## load 'raw' behavioral data

In [36]:
def load_raw_data(exp_name):
    raw_data = pd.read_csv(f'input/raw_data/raw_data_{exp_name}.csv')
    return raw_data

In [37]:
raw_data = load_raw_data(exp_name)

## prepare model inputs

In [38]:
def prep_model_inputs(raw_data):
    subj_encoder = LabelEncoder()
    model_inputs = {
        'N': int(raw_data.shape[0]),
        'J': int(len(set(raw_data.participant))),
        'y': [int(i) for i in raw_data.y],
        'subj': [int(j) for j in subj_encoder.fit_transform(raw_data.participant) + 1],
        'time': list(raw_data.time),
        'x': list(raw_data.err0),
        'gq': int(1),
        'prior_only': int(0),
    }
    return model_inputs, subj_encoder

In [39]:
model_inputs, subj_encoder = prep_model_inputs(raw_data)

### save subject id encoder

In [40]:
def save_subj_encoder(subj_encoder, exp_name):
    with open(f'input/preprocessors/subj_encoder_{exp_name}.pkl', 'wb') as f:
        pickle.dump(subj_encoder, f)
    return f"saved fitted subject id encoder for {exp_name}"

In [41]:
save_subj_encoder(subj_encoder, exp_name)
# NOTE
# to convert encoded subject ids to their original prolific ids:
# subj_encoder.inverse_transform([j-1 for j in model_input['subj'])

'saved fitted subject id encoder for memory'

### save model inputs

In [43]:
def save_model_inputs(model_inputs, exp_name):
    with open(f'input/model_inputs/model_inputs_{exp_name}.json', 'w') as f:
        json.dump(model_inputs, f)
    return f"saved model inputs for {exp_name}"

In [44]:
save_model_inputs(model_inputs, exp_name)

'saved model inputs for memory'

## compile model

In [45]:
model = stan.build(program_code=model_code[model_name], data=model_inputs)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_9e2z7brr/model_wjnrrnvp.stan', line 3, column 26: Declaration
    of arrays by placing brackets after a type is deprecated and will be
    removed in Stan 2.32.0. Instead use the array keyword before the type.
    This can be changed automatically using the auto-format flag to stanc
Warning in '/tmp/httpstan_9e2z7brr/model_wjnrrnvp.stan', line 39, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_9e2z7brr/model_wjnrrnvp.stan', line 40, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
 

## fit model

In [46]:
fit = model.sample(num_samples=2000, num_chains=4)

Sampling:   0%
Sampling:   0% (1/12000)
Sampling:   0% (2/12000)
Sampling:   0% (3/12000)
Sampling:   0% (4/12000)
Sampling:   1% (103/12000)
Sampling:   2% (202/12000)
Sampling:   3% (301/12000)
Sampling:   3% (400/12000)
Sampling:   4% (500/12000)
Sampling:   5% (600/12000)
Sampling:   6% (700/12000)
Sampling:   7% (800/12000)
Sampling:   8% (900/12000)
Sampling:   8% (1000/12000)
Sampling:   9% (1100/12000)
Sampling:  10% (1200/12000)
Sampling:  11% (1300/12000)
Sampling:  12% (1400/12000)
Sampling:  12% (1500/12000)
Sampling:  13% (1600/12000)
Sampling:  14% (1700/12000)
Sampling:  15% (1800/12000)
Sampling:  16% (1900/12000)
Sampling:  17% (2000/12000)
Sampling:  18% (2100/12000)
Sampling:  18% (2200/12000)
Sampling:  19% (2300/12000)
Sampling:  20% (2400/12000)
Sampling:  21% (2500/12000)
Sampling:  22% (2600/12000)
Sampling:  22% (2700/12000)
Sampling:  23% (2800/12000)
Sampling:  24% (2900/12000)
Sampling:  25% (3000/12000)
Sampling:  26% (3100/12000)
Sampling:  27% (3200/12000

## save model and fit

In [47]:
def save_model_fit(model, fit, model_name, exp_name):
    with open(f'output/model_fit/model_fit_{model_name}_{exp_name}.pkl', 'wb') as f:
        pickle.dump({'model' : model, 'fit' : fit}, f, protocol=-1)
    return f"saved {model_name} model and fit for {exp_name}"

In [48]:
save_model_fit(model, fit, model_name, exp_name)

'saved invar_beta model and fit for memory'

# end-to-end loop: all datasets, all models

In [25]:
NUM_SAMPLES = 2000 # number of posterior samples per chain

for exp_name in ALL_EXPERIMENTS:
    for model_name in ALL_MODELS:
        
        print(f'\n\nExperiment: {exp_name}')
        print(f'Model: {model_name}\n\n')
        
        raw_data = load_raw_data(exp_name)
        model_inputs, subj_encoder = prep_model_inputs(raw_data)
        
        save_subj_encoder(subj_encoder, exp_name)
        save_model_inputs(model_inputs, exp_name)

        model = stan.build(program_code=model_code[model_name], data=model_input)
        print("Sampling...")
        fit = model.sample(num_samples=NUM_SAMPLES, num_chains=4)

        save_model_fit(model, fit, model_name, exp_name)



Experiment: memory_color_short
Model: var_beta


Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_hd_8p274/model_rpsbtamc.stan', line 3, column 26: Declaration
    of arrays by placing brackets after a type is deprecated and will be
    removed in Stan 2.32.0. Instead use the array keyword before the type.
    This can be changed automatically using the auto-format flag to stanc
Warning in '/tmp/httpstan_hd_8p274/model_rpsbtamc.stan', line 39, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_hd_8p274/model_rpsbtamc.stan', line 40, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
 

Sampling...



Sampling:   0% (1/12000)
Sampling:   0% (2/12000)
Sampling:   0% (3/12000)
Sampling:   0% (4/12000)
Sampling:   1% (103/12000)
Sampling:   2% (202/12000)
Sampling:   3% (301/12000)
Sampling:   3% (400/12000)
Sampling:   4% (500/12000)
Sampling:   5% (600/12000)
Sampling:   6% (700/12000)
Sampling:   7% (800/12000)
Sampling:   8% (900/12000)
Sampling:   8% (1000/12000)
Sampling:   9% (1100/12000)
Sampling:  10% (1200/12000)
Sampling:  11% (1300/12000)
Sampling:  12% (1400/12000)
Sampling:  12% (1500/12000)
Sampling:  13% (1600/12000)
Sampling:  14% (1700/12000)
Sampling:  15% (1800/12000)
Sampling:  16% (1900/12000)
Sampling:  17% (2000/12000)
Sampling:  18% (2100/12000)
Sampling:  18% (2200/12000)
Sampling:  19% (2300/12000)
Sampling:  20% (2400/12000)
Sampling:  21% (2500/12000)
Sampling:  22% (2600/12000)
Sampling:  22% (2700/12000)
Sampling:  23% (2800/12000)
Sampling:  24% (2900/12000)
Sampling:  25% (3000/12000)
Sampling:  26% (3100/12000)
Sampling:  27% (3200/12000)
Sampling:  2



Experiment: memory_color_short
Model: invar_beta


Building...


...


Sampling:   0% (1/12000)
Sampling:   0% (2/12000)
Sampling:   0% (3/12000)
Sampling:   0% (4/12000)
Sampling:   1% (103/12000)
Sampling:   2% (202/12000)
Sampling:   3% (301/12000)
Sampling:   3% (400/12000)
Sampling:   4% (500/12000)
Sampling:   5% (600/12000)
Sampling:   6% (700/12000)
Sampling:   7% (800/12000)
Sampling:   8% (900/12000)
Sampling:   8% (1000/12000)
Sampling:   9% (1100/12000)
Sampling:  10% (1200/12000)
Sampling:  11% (1300/12000)
Sampling:  12% (1400/12000)
Sampling:  12% (1500/12000)
Sampling:  13% (1600/12000)
Sampling:  14% (1700/12000)
Sampling:  15% (1800/12000)
Sampling:  16% (1900/12000)
Sampling:  17% (2000/12000)
Sampling:  18% (2100/12000)
Sampling:  18% (2200/12000)
Sampling:  19% (2300/12000)
Sampling:  20% (2400/12000)
Sampling:  21% (2500/12000)
Sampling:  22% (2600/12000)
Sampling:  22% (2700/12000)
Sampling:  23% (2800/12000)
Sampling:  24% (2900/12000)
Sampling:  25% (3000/12000)
Sampling:  26% (3100/12000)
Sampling:  27% (3200/12000)
Sampling:  2



Experiment: memory_color_long
Model: invar_beta


Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_9e2z7brr/model_wjnrrnvp.stan', line 3, column 26: Declaration
    of arrays by placing brackets after a type is deprecated and will be
    removed in Stan 2.32.0. Instead use the array keyword before the type.
    This can be changed automatically using the auto-format flag to stanc
Warning in '/tmp/httpstan_9e2z7brr/model_wjnrrnvp.stan', line 39, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_9e2z7brr/model_wjnrrnvp.stan', line 40, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
 

Sampling...



Sampling:   0% (1/12000)
Sampling:   0% (2/12000)
Sampling:   0% (3/12000)
Sampling:   0% (4/12000)
Sampling:   1% (103/12000)
Sampling:   2% (202/12000)
Sampling:   3% (301/12000)
Sampling:   3% (401/12000)
Sampling:   4% (500/12000)
Sampling:   5% (600/12000)
Sampling:   6% (700/12000)
Sampling:   7% (800/12000)
Sampling:   8% (900/12000)
Sampling:   8% (1000/12000)
Sampling:   9% (1100/12000)
Sampling:  10% (1200/12000)
Sampling:  11% (1300/12000)
Sampling:  12% (1400/12000)
Sampling:  12% (1500/12000)
Sampling:  13% (1600/12000)
Sampling:  14% (1700/12000)
Sampling:  15% (1800/12000)
Sampling:  16% (1900/12000)
Sampling:  17% (2000/12000)
Sampling:  18% (2100/12000)
Sampling:  18% (2200/12000)
Sampling:  19% (2300/12000)
Sampling:  20% (2400/12000)
Sampling:  21% (2500/12000)
Sampling:  22% (2600/12000)
Sampling:  22% (2700/12000)
Sampling:  23% (2800/12000)
Sampling:  24% (2900/12000)
Sampling:  25% (3000/12000)
Sampling:  26% (3100/12000)
Sampling:  27% (3200/12000)
Sampling:  2

In [49]:
print("Hello, world!")

Hello, world!
